**Note:** This notebook is written in the Julia language, so the cells can't be
  executed in Google Colab. If you want to verify that the notebook works, I
  recommend [JuliaBox](https://juliabox.com/) or testing locally. The syntax is
  very similar to Python and MATLAB. Note in particular the dot syntax used to
  perform elementwise operations (`f.(x)` applies `f` to all elements of `x`),
  that indices start at 1 and that the last statement of a function is returned
  automatically.

# **Lab 2: Direct methods**
**Anders Ågren Thuné**

# **Abstract**

Short summary of the lab report. State the objectives, methods used, main
results and conlusions.

# *About the code**

A short statement on who is the author of the file, and if the code is
distributed under a certain license.

In [2]:
"""
DD2363 Methods in Scientific Computing,
KTH Royal Institute of Technology, Stockholm, Sweden.
"""

# Copyright (C) 2019
# Anders Ågren Thuné (athune@kth.se)
# Johan Hoffman (jhoffman@kth.se)

# Code written by Anders Ågren Thuné based on the template by Johan Hoffman.

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

"DD2363 Methods in Scientific Computing,\nKTH Royal Institute of Technology, Stockholm, Sweden.\n"

# **Set up environment**

In [3]:
using LinearAlgebra
import Base: getproperty

# **Introduction**

Systems of linear equations appear frequently in a wide variety of problems. A
system of linear equations can be expressed in linear algebra terms as $Ax = b$,
which, if solvable, has the solution $x = A^{-1}b$. As such, being able to
compute the inverse of a given matrix is a problem of great importance. This is
difficult in general, but certain classes of matrices have easily computable
inverses; an orthogonal matrix $Q$ has the inverse $Q^T$ and the inverse of a
triangular matrix can be computed through back (or forward) substitution.
Therefore, a common approach to computing the inverse of a given matrix is to
factor it into triangular and orthogonal matrices, which can then be easily
inverted. One such factorization is the QR factorization in which any matrix $A$
is factored into an orthogonal matrix $Q$ and an upper triangular matrix $R$.
The inverse is then given by $A^{-1}=(QR)^{-1}=R^{-1}Q^T$.

Even when the system $Ax=b$ is unsolvable (there is no $A^{-1}), the best
possible solution can be obtained by projecting $b$ orthogonally onto
$range(A)$. This gives the approximated solution $\hat{x} = (A^TA)^{-1}A^Tb$,
where $(A^TA)^{-1}$ is called the *pseudo-inverse* of $A$.

The QR factorization can also be used in a wider range of problems. One such
problem is eigenvalue computation, where an iterative $QR$-algorithm can be used
to obtain a Schur factorization of a symmetric matrix. That is, a factorization
$A = QTQ^T$, where $T$ is a triangular matrix with the eigenvalues of $A$ on
the diagonal.

This report presents how these concepts, described in chapters 5 and 6 of the
lecture notes, were used to implement the following:
- A function for QR factorization
- A direct solver of of $Ax=b$
- A solver of the least squares problem $Ax=b$
- A function performing the QR eigenvalue algorithm

# **Methods**

## **QR factorization**

The QR factorization for a matrix can be calculated in a number of different
ways. The method implemented here is the *Householder QR factorization*, based
on Householder reflections. A matrix $P$ of the form $P = I-\beta vv^T, \quad
\beta = \frac{2}{v^Tv}$ is a *Householder reflection*. This is an orthogonal
reflector reflecting a given vector $x$ in the hyperplane
$\text{span}\{v\}^\perp. Selecting $v$ = $\pm \|x\|$ gives $Px = \pm\|x\|e_1$,
which can be utilized to construct $Q_n\dots Q_2Q_1A = R$, where each matrix
$Q_k$ is constructed to zero the subdiagonal elements of a column of $A$. This
is achieved by letting $Q_k = \begin{matrix} I & 0 \\ 0 P \end{matrix}$, where P
is a Householder reflection. When dealing with a Householder QR factorization,
it is advantageous not to explicitly form the matrices $Q$ or $Q_k$, but instead
utilize that $Q_kA = A - (\beta v)(v^TA)$ and $Q_kA = A - (A v)(\beta
v)^T$.

[Golub & Van Loan 2013: Chapter 5.1-5.2, Hoffman 2019: Chapter 5.2]

The following implementation is based on Algorithm 5.2.1 of Golub & Van Loan
(2013), and results in a matrix with the upper triangular part of $R$ as its
upper triangular part, and the vectors $v_j$ required to construct $Q$ below the
diagonal. To be precise, all elements except the first of these vectors are
stored, with the implicit assumption that they are normalized such that the
first element is always 1. In addition to the function itself, a wrapper
struct (analogous to a Python class) is implemented to facilitate easy handling
of this format.

In [445]:
"""
A struct for representing the QR factorization of a matrix
in a compact way. The constructor computes the factorization
using Algorithm 5.2.1 of Golub & Van Loan (2013),
modifying the argument in place.
"""
mutable struct QRfact{T <: AbstractFloat}
    QR :: Matrix{T}
    betas :: Vector{T}

    function QRfact(A :: Matrix{T}) where T <: AbstractFloat
        (m, n) = size(A)
        v = zeros(T, m)
        betas = zeros(T, n)

        for j = 1 : n
            v[j:m] .= A[j:m,j]
            v[j] += sign(v[j])*norm(v[j:m])
            betas[j] = 2v[j]^2/norm(v[j:m])^2
            v[j:m] ./= v[j]
            for k = j : n
                A[j:m,k] .-= betas[j].*v[j:m].*dot(v[j:m],A[j:m,k])
            end
            if j<m
               A[j+1:m,j] .= v[j+1:m]
            end
        end
        new{T}(A, betas)
    end
end

"""
Overloads getproperty to allow easy extraction of Q and R factors.
Enables dot syntax, as in the following example:
> QR = QRfactorization(A)
> Q = QR.q
> R = QR.r
"""
function getproperty(qr :: QRfact{T}, f :: Symbol) where T
    QR = getfield(qr, :QR)
    (m, n) = size(QR)
    if f == :q
        Q = Matrix{T}(I,m,m)
        mulQ!(qr,Q)
    elseif f == :qt
        Qt = Matrix{T}(I,m,m)
        mulQ!(qr,Qt,transpose=true)
    elseif f == :r
        triu(QR)
    elseif f == :factors
        QR
    else
        error("type $(typeof(qr)) has no field $f")
    end
end

"""
Multiply the matrix C with the Q factor of a QR factorization.
Keywords specify whether to multiply with Q or Q^T, and whether
to multiply from the left or from the right. Modifies C in place
and returns the result.
"""
function mulQ!(qr :: QRfact{T}, C :: VecOrMat{T};
               transpose = false, fromright = false) where T
    QR = getfield(qr, :QR)
    betas = getfield(qr, :betas)
    (m,n) = size(QR)
    Csize = typeof(C) <: Vector ? (length(C), 1) : size(C)

    index_to_match = Int(fromright)+1
    if Csize[index_to_match] != m
        throw(DomainError((m,Csize[index_to_match]),
            "The matrices' dimensions don't match!"))
    end
    iterateForwards = xor(fromright, transpose)
    iterRange = iterateForwards ? (1:n) : (n:-1:1)

    v = zeros(T, m)
    temp = zeros(T,Csize[1])
    for j = iterRange
        v[j] = one(T)
        v[j+1:m] .= QR[j+1:m,j]
        if fromright
            mul!(temp, C[:,j:m], v[j:m])
            for k = j : m
                C[:,k] .-= temp .*betas[j].*v[k]
            end
        else
            for k = 1 : Csize[2]
                C[j:m,k] .-= betas[j].*v[j:m].*dot(v[j:m],C[j:m,k])
            end
        end
    end
    C
end

"""
Get the QR factorization of A without modifying the original.
"""
QRfactorization(A :: Matrix{<:AbstractFloat}) = QRfact(copy(A))

QRfactorization

## **Direct solver**

The solution to the system $Ax=b$ can be obtained as $x = A^{-1}b = R^{-1}Q^Tb$,
once $A$ has been factored into $Q$ and $R$. Setting $y$ = $Q^Tb$, the equation
to solve is $x = R^{-1}y$. Algorithm 2 of chapter 5 in the notes describes how
such a system can be solved, using *back substitution*. In the implementation
below, the summation of the original algorithm is replaced by a dot product.

In [466]:
"""
Compute x = U^-1 b through back substitution for any upper triangular matrix U.
"""
function backsubstitute(U::Matrix, b::Vector)
    m = length(b)
    if m != size(U)[1]
        throw(DomainError((m,size(U)[1]), "The dimensions of U and b don't match!"))
    end
    x = zeros(m)
    x[m] = b[m]/U[m,m]
    for j = m-1:-1:1
        x[j] = (b[j] - dot(U[j,j+1:m],x[j+1:m]))/U[j,j]
    end
    x
end

backsubstitute

In [467]:
"""
Compute x = A^-1 b for any invertible matrix A.
"""
function directsolver(A::Matrix, b::Vector)
    qr = QRfactorization(A)
    y = copy(b)
    mulQ!(qr,y, transpose=true)
    backsubstitute(qr.factors,y)
end

directsolver

## **Least squares solver**

This is very similar to the above. In order to solve the least squares problem,
we solve $x = (A^TA)^{-1}(A^Tb)$, which is an equation solvable directly, as
$A^TA$ is a positive definite symmetric matrix for all $A$.

In [468]:
"""
Compute the best solution x to Ax = b for a non-invertible matrix A, given by
x = (A^T A)^-1 (A^T b)
"""
leastsquares(A::Matrix, b::Vector) = directsolver(A'A, A'b)

leastsquares

In [469]:
A = rand(6,3)

6×3 Array{Float64,2}:
 0.0322851  0.602584   0.00667616
 0.952147   0.431032   0.950972  
 0.385927   0.0360104  0.816445  
 0.600568   0.379458   0.805187  
 0.248099   0.886512   0.311007  
 0.845026   0.0967036  0.0485464 

In [470]:
b = rand(6)

6-element Array{Float64,1}:
 0.7910663988052073 
 0.279495963495076  
 0.05642527638757877
 0.8857219521340596 
 0.5982403158058058 
 0.9692496834761894 

In [471]:
test = qr(A)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
6×6 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.021802   0.572056     0.163965   -0.259826   -0.750748   -0.119415 
 -0.64298    0.00818856  -0.181365   -0.282395    0.188352   -0.662112 
 -0.260614  -0.131558    -0.601829   -0.455626   -0.156478    0.566122 
 -0.405561   0.109787    -0.324557    0.80254    -0.263822    0.0667638
 -0.16754    0.755015     0.0654419  -0.0076105   0.547305    0.313049 
 -0.570642  -0.270697     0.684401   -0.0319328  -0.0852688   0.352698 
R factor:
3×3 Array{Float64,2}:
 -1.48083  -0.65727  -1.23074 
  0.0       1.02832   0.214268
  0.0       0.0      -0.870489

In [472]:
f = getfield(test.Q, :factors)

6×3 Array{Float64,2}:
 -1.48083   -0.65727    -1.23074  
  0.629261   1.02832     0.214268 
  0.255054   0.205257   -0.870489 
  0.396907   0.0867496   0.230218 
  0.163965  -0.489144    0.0634757
  0.558466   0.436587   -0.452333 

In [473]:
test2 = QRfactorization(A)
getfield(test2, :QR)

6×3 Array{Float64,2}:
 -1.48083   -0.65727    -1.23074  
  0.629261   1.02832     0.214268 
  0.255054   0.205257   -0.870489 
  0.396907   0.0867496   0.230218 
  0.163965  -0.489144    0.0634757
  0.558466   0.436587   -0.452333 

In [474]:
r = test2.r

6×3 Array{Float64,2}:
 -1.48083  -0.65727  -1.23074 
  0.0       1.02832   0.214268
  0.0       0.0      -0.870489
  0.0       0.0       0.0     
  0.0       0.0       0.0     
  0.0       0.0       0.0     

In [475]:
r[1]=3

3

In [476]:
getfield(test2, :QR)

6×3 Array{Float64,2}:
 -1.48083   -0.65727    -1.23074  
  0.629261   1.02832     0.214268 
  0.255054   0.205257   -0.870489 
  0.396907   0.0867496   0.230218 
  0.163965  -0.489144    0.0634757
  0.558466   0.436587   -0.452333 

In [477]:
A

6×3 Array{Float64,2}:
 0.0322851  0.602584   0.00667616
 0.952147   0.431032   0.950972  
 0.385927   0.0360104  0.816445  
 0.600568   0.379458   0.805187  
 0.248099   0.886512   0.311007  
 0.845026   0.0967036  0.0485464 

In [478]:
test2.q*r

6×3 Array{Float64,2}:
 -0.0654059  0.602584   0.00667616
 -1.92894    0.431032   0.950972  
 -0.781843   0.0360104  0.816445  
 -1.21668    0.379458   0.805187  
 -0.502619   0.886512   0.311007  
 -1.71193    0.0967036  0.0485464 

In [479]:
Ass = copy(A)
mulQ!(test2, Ass, transpose=true)
Ass

6×3 Array{Float64,2}:
 -1.48083      -0.65727      -1.23074    
 -1.77219e-16   1.02832       0.214268   
 -1.25326e-17   2.63808e-17  -0.870489   
  7.11934e-17   4.97521e-18   0.0        
  1.92416e-16   1.1622e-16   -1.38778e-17
  1.25329e-16  -3.70424e-17   0.0        

In [480]:
testerrand = rand(3, 6)

3×6 Array{Float64,2}:
 0.585429  0.826954   0.52445   0.645115  0.924462  0.322732 
 0.679941  0.747981   0.102294  0.176977  0.559523  0.489632 
 0.246689  0.0551864  0.957945  0.686303  0.301666  0.0920479

In [481]:
testerrand * test.Q'

3×6 Array{Float64,2}:
 -0.353903   -0.686502  -0.832877  -0.0214679  1.16268   -0.184591
 -0.0946776  -0.718398  -0.228166  -0.199733   0.915676  -0.401138
 -0.232526   -0.529837  -0.955862   0.0724491  0.251722   0.484735

In [482]:
res = copy(testerrand)
mulQ!(test2,res,fromright=true,transpose=true)
res

3×6 Array{Float64,2}:
 -0.353903   -0.686502  -0.832877  -0.0214679  1.16268   -0.184591
 -0.0946776  -0.718398  -0.228166  -0.199733   0.915676  -0.401138
 -0.232526   -0.529837  -0.955862   0.0724491  0.251722   0.484735

In [483]:
A\b

3-element Array{Float64,1}:
  0.9003040935948777
  0.8238704177869213
 -0.5285491649657518

In [484]:
leastsquares(A,b)

3-element Array{Float64,1}:
  0.9003040935948778
  0.823870417786922 
 -0.5285491649657525

# **Results**

Present the results. If the result is an algorithm that you have described under
the *Methods* section, you can present the data from verification and
performance tests in this section. If the result is the output from a
computational experiment this is where you present a selection of that data.

# **Discussion**

Summarize your results and your conclusions. Were the results expected or
surprising. Do your results have implications outside the particular problem
investigated in this report?

# **References**

- Hoffman, J. 2019. *Introduction to Scientific Computing*
- Golub, Gene H. and Van Loan, Charles F. 2013. *Matrix Computations*. 4th ed. Baltimore: John Hopkins University Press.